In [13]:
import os
import re
import sys
import Levenshtein
import openai
import torch
import torch.nn as nn
import math
import copy
import time
import pandas as pd
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
import torch.distributed as dist
import torch.multiprocessing as mp


from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer, GPT2Tokenizer, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from nltk.tokenize import sent_tokenize, LineTokenizer
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from pathlib import Path
from torch.nn.functional import log_softmax, pad
from Levenshtein import distance
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from translate.storage.tmx import tmxfile


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-de"
translator = pipeline("translation", model=model_checkpoint)
translator("How are you?")

[{'translation_text': 'Wie geht es dir?'}]

In [9]:
with open("de-en.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'de')
file_de = open("de_origin/de.txt",'w', encoding="utf-8")
file_en = open("en_origin/en.txt",'w', encoding="utf-8")

for node in tmx_file.unit_iter():
    file_en.write(node.target + '\n')
    file_de.write(node.source + '\n')
    # print(node.target, node.source)
    
file_de.close()
file_en.close()

In [10]:
with open("en_origin/en.txt", encoding="utf-8") as myfile:
    head = [next(myfile) for x in range(10)]
print(head)

with open("de_origin/de.txt", encoding="utf-8") as myfile:
    head = [next(myfile) for x in range(10)]
print(head)

['ANNUAL ACTIVITY REPORT 2007 EUROPEAN COURT OF AUDITORS \n', 'Luxembourg: Office for Official Publications of the European Communities, 2008 ISBN 978-92-9207-005-2 © European Communities, 2008 Reproduction is authorised provided the source is acknowledged. \n', '3 4–5 6–7 8–13 14–17 18–25 26–27 28–29 30–35 36–37 38–43 44–45 PRESIDENT’S FOREWORD MISSION, VISION, VALUES AND STRATEGIC OBJECTIVES THE COURT’S ROLE AND WORK GOVERNANCE AND ORGANISATION OVERVIEW OF AUDIT REPORTS AND OPINIONS FOLLOW-UP AND IMPACT THE COURT’S VIEW THE COURT’S WORK IN 2007 AND BEYOND INTERNATIONAL COOPERATION HUMAN RESOURCES FINANCIAL INFORMATION CONTENTS \n', 'PRESIDENT’S FOREWORD 4 Bringing the European Court of Auditors closer to EU citizens is one of our key objectives and part of our mission to promote transparency and accountability. \n', 'I, therefore, take great pleasure in welcoming you to the first annual activity report of the European Court of Auditors. \n', 'It provides an overview of the Court and 

In [11]:
with open("de-en.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'de')
file_de = open("de_origin/de_10000.txt",'w', encoding="utf-8")

i = 0
for node in tmx_file.unit_iter():
    if(i<10000):
        file_de.write(node.source + '\n')
    i+=1
    
file_de.close()

In [15]:
#model_checkpoint = "facebook/mbart-large-50-many-to-one-mmt"

model_checkpoint = "Helsinki-NLP/opus-mt-de-en"

#tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)
#model = MarianMTModel.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
 
dev = "cuda:0"
device = torch.device(dev)
model.to(device)

#na jednym z dodatkowymi parametra
#tłumaczenie przez język pośredni

args = [
        'model.generate(**model_inputs)', 
        'model.generate(**model_inputs, do_sample=True, top_k=0)',
        'model.generate(**model_inputs, do_sample=True, top_k=0, temperature=1)',
        'model.generate(**model_inputs, do_sample=True, top_k=50)',
        'model.generate(**model_inputs, do_sample=True, top_k=6)',
        'model.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.40)',
        'model.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92)',
        'model.generate(**model_inputs, do_sample=True, top_k=0, repetition_penalty=0.5)',
        'model.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92, repetition_penalty=0.5)'
       ]
titles = [
          'no_params',
          'sample',
          'temperature', 
          'top_k50',
          'top_k6',
          'top_p40',
          'top_p92',
          'repetition_penalty05',
          'repetition_penalty_top_k6'
         ]

for arg,title in zip(args,titles):
    print(arg, title)

    file = open("Translation\Auto_"+title+".txt",'w', encoding="utf-8")
    with open("de-en.tmx", 'rb') as fin:
        tmx_file = tmxfile(fin, 'en', 'de')

    i = 0
    for node in tmx_file.unit_iter():
        if i == 10000:
            break

        model_inputs = tokenizer(node.source, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        translated = eval(arg)
        for t in translated:
            if(i%100==0):
                print(repr(i) + " " + tokenizer.decode(t, skip_special_tokens=True) + '\n')
            file.write(tokenizer.decode(t, skip_special_tokens=True) + '\n') 
        i+=1


    file.close()

model.generate(**model_inputs) no_params
0 ANNUAL ACTIVITY REPORT 2007 EUROPEAN COURT OF AUDITORS

100 As regards the areas of expenditure identified in the ‘Final accounts of the European Communities' in which most payments are made – agriculture [mainly agricultural expenditure not covered by the Integrated Administration and Control System (IPCS)], structural measures, internal policies and large parts of external policies – the Court gives a negative opinion on legality and regularity and concludes that these areas of expenditure continue to be significantly affected by errors.

200 National declarations may also contain certain audit opinions on the legality and regularity of the transactions.

300 The audit teams usually consist of two to three examiners and the length of an examination trip may be up to two weeks depending on the type of examination and the distance.

400 9 OVERVIEW OF THE INTERNAL AND EXTERNAL CONTROLS OF THE EU BUDGET EU level Internal control External control

3400 In addition, they consulted relevant stakeholders in the areas of food production, retail trade, research and charities. "The EU can and should do more to combat food waste effectively," said EU auditors The auditors concluded that while food waste could be combated through a number of EU policies, this potential is not exploited.

3500 Our strategy for 2018-2020: strengthening confidence through external financial control In July 2017, we published our strategy for the period 2018-2020, in which we set four strategic objectives: • communicating clear statements to our audience; • focusing more on the performance aspects of EU actions; • improving the added value of the Statement of Assurance; • aligning our overall organisation with our products.

3600 A detailed study of this project will be carried out in 2018 and work is expected to begin in 2019.

3700 Acceptance of the report by the Chamber of Examination or the whole College of Court.

3800 We gave o full audit opinions on 

7700 3.11.10.

7800 EN C 434/90 Official Journal of the European Union 30.11.2018 3.14.

7900 Therefore, Eurofound will continue to follow developments and is ready to invest in electronic procurement once it is satisfied with its stability and functionality.

8000 In several procurement procedures, some angels have been identified.

8100 In accordance with its framework financial regulation, the agencies should establish a uniform framework for electronic storage and exchange of data with third parties involved in procurement procedures (160) (161). As a requirement also applies to the EU institutions, the Commission is currently developing an overall IT system covering all stages of public procurement procedures. It launched tools for electronic invoicing (2010), the electronic publication of documents on contract notices in the Official Journal of the EU (2011) and the electronic submission of tenders (2015).

8200 The personnel-related payments amounted to 13.5 million euros in 201

200 National declarations may also contain certain examination judgments on the legality and regularity of transactions.

300 The audit teams usually consist of two to three auditors, and the length of an examination trip may be up to two weeks depending on the type of examination and the distance.

400 9 OVERVIEW OF THE INTERNAL AND EXTERNAL CONTROLS OF THE EU BUDGET EU level Internal control External control Commission (DGs, Internal Audit Service) European Court of Auditors Member States Implementing Bodies National control authorities National level

500 If the agencies had sufficiently planned their actions, had they set up reliable monitoring tools for their activities and adequately reported on their activities, and conducted sound assessments of their results? (No 5/2008) 24

600 Four key performance indicators (1 to 4) are used to measure the impact of the Court's work.

700 In October 2008, the budgetary authority approved a tranche of EUR 55 million for the 2009 budget.

800

3600 A detailed study of this project will be carried out in 2018 and work is expected to begin in 2019.

3700 Acceptance of the report by the Chamber of Examination or the whole College of Court.

3800 We gave o full audit opinions on the reliability of the accounts for all agencies; o full audit opinions on the legality and regularity of the revenue underlying the annual accounts of all agencies; o full audit opinions on the legality and regularity of the payments underlying the annual accounts of all agencies, with the exception of a failed audit opinion for the European Asylum Support Office, which identified a number of problems.

3900 Performance measurement Our new key performance indicators as part of the 2018-2020 strategy Since 2008, we have been using Key Performance Indicators (KPI) to inform management of progress made towards our goals, support the decision-making process, and provide information about our performance to our addressees.

4000 As part of our goal of system

...

7400 The table contains the most important figures for the survey (77).

7500 The EIT can therefore offer temporary temporary workers temporary employment with a maximum duration of five years which can be extended by five years.

7600 At EIT Digital, this share was 0.6%.

7700 3.11.10.

7800 EN C 434/90 Official Journal of the European Union 30.11.2018 3.14.

7900 Eurofound will therefore continue to follow developments and is ready to invest in electronic procurement once it is satisfied with its stability and functionality.

8000 Several proposals were found in several award procedures.

8100 In its Framework Financial Regulation, the agencies should establish a single arrangement for the electronic storage and exchange of data with third parties involved in procurement procedures (160) (161). As for the EU institutions, the Commission is in the process of developing a comprehensive electronic information exchange with third parties involved in procurement procedures. It launched to

9900 C 373/34 Official Journal of the European Union 10.11.2015 REPLY OF THE COMMISSION OF THE COMMENTS OF THE HOFE... while net corrections are made less frequently 1.41.

model.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92, repetition_penalty=0.5) repetition_penalty_top_k6
0 ANNUAL ACTIVITY REPORT 2007 EUROPEAN COURT OF AUDITORS

100 As regards the areas of expenditure identified in the ‘Final accounts of the European Communities' in which most payments are made – agriculture [mainly agricultural expenditure not covered by the Integrated Administration and Control System (IPCS)], structural measures, internal policies, and large parts of external policies – the Court gives a negative opinion on the legality and regularity of the accounts and concludes that these areas of expenditure continue to be significantly affected by errors.

200 National declarations may also contain certain audit opinions on the legality and regularity of the transactions.

300 The audit teams 

3400 They also consulted relevant stakeholders in the areas of food production, retail trade, research and charities. "The EU can and should do more to combat food waste effectively," said the EU auditors. "The auditors concluded that while food waste can and should be combated through a number of EU policies, this potential is not exploited.

3500 Our strategy for 2018-2020: strengthening confidence through external financial control In July 2017, we published our strategy for the period 2018-2020, in which we set four strategic goals: • communicating clear statements to our audience; • focusing more on the performance aspects of EU actions; • improving the added value of the Statement of Assurance; • focusing our overall organisation on our products.

3600 A detailed study of this project will be carried out in 2018 and work is expected to begin in 2019.

3700 Acceptance of the report by the Chamber of Examining or the entire College of Court.

3800 We gave full audit opinions on the

6700 EN C 434/8 Official Journal of the European Union 30.11.2018 The Commission's executive agencies run EU programmes by 1.10.

6800 The Court also pointed out the impact of the current London lease agreements and the possible decline in revenues resulting from the United Kingdom's withdrawal from the EU.

6900 In accordance with the Financial Regulation, the auditors must confirm that the agencies' accounting systems function reliably (validation).

7000 3.1.8.

7100 In December 2017, the B thero signed legal commitments and commitments of EUR 20 000 for a training programme planned for the end of March 2018.

7200 (53) The budget implementation reports include the overviews that summarize all revenue and expenditure forecasts, as well as explanations.

7300 In carrying out procurement procedures, the EBA strives to use all the tools available to it to maximize competition.

7400 The table contains the most important figures for the survey (77).

7500 The EIT can therefore only offe

9700 10.11.2015 Official Journal of the European Union REPLY OF THE COMMISSION C 373/17 DE COMMISSION REPLY C 373/17 DE COMMISSION REPLY OF THE HOFES Regularity of transactions: Although our general examination results are stable, they indicate a material error rate 1.9.

9800 The Commission considers that, overall, IACS is a sound system for managing CAP expenditure.

9900 C 373/34 Official Journal of the European Union 10.11.2015 REPLY OF THE COMMISSION OF THE COMMENTS OF THE HOFES... while net corrections are less frequent 1.41.



In [73]:
total = 10000
score = 0

os.chdir('C:/Users/salan/Studia/translation')

file_org = open("En_origin\en.txt",'r+', encoding="utf-8")
file_eu = open("En_origin\en_EU.txt",'r+', encoding="utf-8")
file_google = open("En_origin\en_google_10000.txt",'r+', encoding="utf-8")

os.chdir('C:/Users/salan/Studia/translation/Translation')

for x,filename in enumerate(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r+', encoding="utf-8") as results:
        for i in range (total):
            reference1 = (file_org.readline().split())
            reference2 = (file_eu.readline().split())
            reference3 = (file_google.readline().split())

            translated = (results.readline().split())

            score += sentence_bleu([reference1,reference2,reference3], translated)

        print(filename + ": BLEU = " + repr(score/total))

    results.close()
file_org.close()
file_eu.close()
file_google.close()

Auto_no_params.txt: BLEU = 0.6561369062959935
Auto_repetition_penalty05.txt: BLEU = 0.6576432436464936
Auto_repetition_penalty_top_k6.txt: BLEU = 0.6583266514428858
Auto_sample.txt: BLEU = 0.6586756448975135
Auto_temperature.txt: BLEU = 0.6589852596651515
Auto_top_k50.txt: BLEU = 0.6593715489572654
Auto_top_k6.txt: BLEU = 0.660687300237249
Auto_top_p40.txt: BLEU = 0.6616250205163312
Auto_top_p92.txt: BLEU = 0.6623405419410194


In [71]:
model_checkpoint_pl = "Helsinki-NLP/opus-mt-de-pl"
model_checkpoint_en = "Helsinki-NLP/opus-mt-pl-en"

tokenizerPl = MarianTokenizer.from_pretrained(model_checkpoint_pl)
modelPl = MarianMTModel.from_pretrained(model_checkpoint_pl)

tokenizerEn = MarianTokenizer.from_pretrained(model_checkpoint_en)
modelEn = MarianMTModel.from_pretrained(model_checkpoint_en)
 
dev = "cuda:0"
device = torch.device(dev)
modelPl.to(device)
modelEn.to(device)

#na jednym z dodatkowymi parametra
#tłumaczenie przez język pośredni

os.chdir('C:/Users/salan/Studia/translation')

argsPl = [
        'modelPl.generate(**model_inputs)', 
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=0)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=0, temperature=1)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=50)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=6)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.40)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=0, repetition_penalty=0.5)',
#        'modelPl.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92, repetition_penalty=0.5)'
       ]
argsEn = [
        'modelEn.generate(**model_inputs)', 
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=0)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=0, temperature=1)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=50)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=6)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.40)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=0, repetition_penalty=0.5)',
#        'modelEn.generate(**model_inputs, do_sample=True, top_k=6, top_p=0.92, repetition_penalty=0.5)'
       ]
titles = [
          'no_params',
#          'sample',
#          'temperature', 
#          'top_k50',
#          'top_k6',
#          'top_p40',
#          'top_p92',
#          'repetition_penalty05',
#          'repetition_penalty_top_k6'
         ]


for i in range(len(titles)):
    print(argsPl[i], titles[i])

    file = open("TranslationTwoLanguages\Marian_"+titles[i]+".txt",'w', encoding="utf-8")
    with open("de-en.tmx", 'rb') as fin:
        tmx_file = tmxfile(fin, 'en', 'de')

    x = 0
    for node in tmx_file.unit_iter():
        if x == 5:
            break

        model_inputs = tokenizerPl(node.source, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        translatedPl = eval(argsPl[i])
        
        textPl = ''
        for t in translatedPl:
            textPl += tokenizerPl.decode(t, skip_special_tokens=True)
            file.write(tokenizerPl.decode(t, skip_special_tokens=True) + '\n') 
        print(node.source)
        print(textPl)
        
        model_inputs = tokenizerEn(textPl, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        translatedEn = eval(argsEn[i])
        
        for t in translatedEn:
            print( tokenizerEn.decode(t, skip_special_tokens=True) + '\n')
            file.write(tokenizerEn.decode(t, skip_special_tokens=True) + '\n') 
        x+=1


    file.close()

modelPl.generate(**model_inputs) no_params
JÄHRLICHER TÄTIGKEITSBERICHT 2007 EUROPÄISCHER RECHNUNGSHOF 
Roczne SPRAWOZDANIE z DZIAŁALNOŚCI 2007 EUROPEJSKI TRYBUNAŁ OBRACHUNKOWY
Annual report on the operation of the European Court of Auditors 2007

Luxemburg: Amt für amtliche Veröffentlichungen der Europäischen Gemeinschaften, 2008 ISBN 978-92-9207-003-8 © Europäische Gemeinschaften, 2008 Nachdruck mit Quellenangabe gestattet. 
Luksemburg: Urząd Oficjalnych Publikacji Wspólnot Europejskich, 2008 ISBN 978-92-9207-003-8 © Wspólnoty Europejskie, 2008 Powielanie dozwolone pod warunkiem podania źródła.
Luxembourg: Office for Official Publications of the European Communities, 2008 ISBN 978-92-9207-003-8 © European Communities, 2008 Reproduction is authorised provided the source is acknowledged.

3 4–5 6–7 8–13 14–17 18–25 26–27 28–29 30–35 36–37 38–43 44–45 VORWORT DES PRÄSIDENTEN AUFTRAG, LEITBILD, WERTE UND STRATEGISCHE ZIELE ROLLE UND ARBEIT DES HOFES LEITUNGSSTRUKTUR UND ORGANISATION ÜBER